In [15]:
# BLS code formats: https://www.bls.gov/help/hlpforma.htm#WP
# PPI codes downloads: https://www.bls.gov/ppi/data-retrieval-guide/
# PPI weights, seasonal adjustments: https://www.bls.gov/ppi/tables/commodity-special-requests.htm
# manual data requests: https://data.bls.gov/series-report


In [23]:
#IMPORT DATA
%reset -f
import pandas as pd
codes_backup= pd.read_csv("PPI_commodity_codes.csv")

codes_backup.columns=["identity","content"]


#ID CONTENT:
#wpu10-80: "commodities/services". Headline PPI based on these. 5th digit gives group, 6-7th gives item. Varied length for most disaggregated.
#wpuIP: "inputs to industry", where industries are based on NAICS classification (like in other spreadsheet). Note that more complete (but experimental) version exists: https://www.bls.gov/ppi/input-indexes/
#wpuFD: "final demand": goods grouped by the final consumer. 
#wpuID51-54: "intermediate demand": four stages of production (7 digits), with corresponding inputs (lowest most interesting).
#wpuID59: intermediate demand total good/service/food/energy etc. inputs to each stage (all 10 digits).
#wpu61-69: various inputs (materials, components, supplies, transportation) for different types of intermediate processing (manufacturing, construction, etc.)

#wpd: discontinued.

In [25]:
#CLEAN DATA, BASICS

#Choose seasonally adjusted. ISSUE: only a few series are seasonally adjusted. BLS has test and most products in most years have too many other factors impacting them for obvious seasonal trends. 
#df['identity'] = df.identity.str.replace('wpu', 'wps')

#PICK COMMODITIES/SERVICES.
#select commodities/services indexes. Based on 4th character being a number between 1-15 or 30-80.
codes_com = codes_backup[codes_backup["identity"].str[3].str.isnumeric()]

#select intermediate demand-final demand indexes. Based on 4th-5th characters being FD or ID.
#codes_idfd = codes_backup[codes_backup["identity"].str.startswith(("wpuFD", "wpuID"))]


#ID CONTENT:
#wpu10-80: "commodities/services". Headline PPI based on these(?). 5th digit gives group, 6-7th gives item. Varied length for most disaggregated.
#wpuIP: "inputs to industry", where industries are based on NAICS classification (like in other spreadsheet). Note that more complete (but experimental) version exists: https://www.bls.gov/ppi/input-indexes/
#wpuFD: "final demand": goods grouped by the final consumer. 
#wpuID51-54: "intermediate demand": four stages of production (7 digits), with corresponding inputs (lowest most interesting).
#wpuID59: intermediate demand total good/service/food/energy etc. inputs to each stage (all 10 digits).
#wpuID61-69: various inputs (materials, components, supplies, transportation) for different types of intermediate processing (manufacturing, construction, etc.)



In [41]:
#DATA CLEANING: select which levels of aggregation to use.

#create new column of just the numeric part of the codes
codes_com['code'] = codes_com.identity.str.extract('(\d+)')

codes_com['code_length'] = codes_com.code.str.len()

codes_com.groupby('code_length').size()
# code_length
# 2       85
# 3      265
# 4      429
# 6      816
# 7       28
# 8     1420
# 9      464
# 10     110
# 11      15

#Even-digit codes (2,4,6,etc.) should add up to 100% of overall index and not overlap. In-between are special case indexes.
#6-digit codes are 'baseline' on which other indexes (inc. FD-ID?) are based.
#>6-digit codes some issues appear because they'll include products which are used at lots of stages of production/industries. Gets messy.
#<6-digit are products which go into 6-digit codes.


/var/folders/wq/93tz19ss0tb00nycxgx5d08h0000gn/T/ipykernel_4668/1556985213.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  codes_com['code'] = codes_com.identity.str.extract('(\d+)')
/var/folders/wq/93tz19ss0tb00nycxgx5d08h0000gn/T/ipykernel_4668/1556985213.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  codes_com['code_length'] = codes_com.code.str.len()


In [ ]:
codes_com = codes_com[(codes_com['code_length']==2) | (codes_com['code_length']==6)]

In [ ]:
#DEFINE FUNCTION FOR MAKING API REQUEST

import requests
import json
from datetime import datetime

def import_BLS_data(codes_list, start, end):

    def chunker(data, length):  # Split list into chunks of equal size. For dealing with API max requests.
        chunk_list = []  # Define list of chunks
        for i in range(0, len(data), length):  # Iterate in steps.
            chunk_list.append(data[i:i+length])  # Add each step as a separate list.
        return chunk_list  # Return list of lists.

    # API endpoint
    url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
    series_id_list = list(codes_list["identity"])
    API_KEY = 'a54f3f2fc04f4c548e347be4e76f528f'
    start_year = int(start)
    end_year = int(end)
    
    # Iterate over the specified periods in chunks (10 years at a time)
    for start_date in range(start_year, end_year, 10):
        end_date = min(start_date + 9, end_year)
        for chunk in chunker(series_id_list, 50):
            chunk_list = list(chunk)
            headers = {'Content-type': 'application/json'}
            data = json.dumps({
                "seriesid": chunk_list,
                "startyear": str(start_date),
                "endyear": str(end_date),
                "registrationkey": API_KEY
            })
            response = requests.post(url, data=data, headers=headers)
            json_data = json.loads(response.text)

            # Loop through each series in the returned data
            for series in json_data['Results']['series']:
                series_id = series['seriesID']
                series_points = series['data']

                # Parse the dates and values for this series
                for point in series_points:
                    date_str = f"{point['year']}-{point['period'][1:]}"
                    date = datetime.strptime(date_str, "%Y-%m").strftime('%Y-%m')
                    value = float(point['value'])

                    # Check if the column (date) already exists in the dataframe, otherwise create it
                    if date not in codes_list.columns:
                        codes_list[date] = None  # Add the date column

                    # Find the index of the series_id in codes_list
                    row_index = codes_list[codes_list['identity'] == series_id].index[0]

                    # Assign the value directly to the corresponding row and column
                    codes_list.at[row_index, date] = value

    return codes_list

In [197]:
#MAKE REQUEST AND BACKUP
commodity_df = import_BLS_data(codes_com[['identity']], 2000, 2024)
commodity_backup = commodity_df

In [ ]:
commodity_df.to_csv('PPI_data.csv')

In [107]:
# #CLEAN DATA: create functions to select different levels of aggregation (industries, categories, lowest).
# #function for selecting highest level (industries).

# #function for selecting a level of disaggregation.
# def one_level(df, column_name, level):
#     df_industry = df.loc[df.identity.str.len()==int(level)]
#     df_industry = df_industry.reset_index(drop=True)
#     return df_industry

# # #DROPPED
# # #function for selecting an intermediate level of aggregation (categories or products).
# # def intermediate_level(df, column_name, level):
# #     df_intermediate = df.loc[df[column_name].str.len().isin([level,level+1])].reset_index(drop=True) #create dataframe where codes are of intermediate length (e.g. 5 or 6 for categories etc.) 
# #     rows_to_drop= set()

# #     #sort to make sure related indexes are grouped.
# #     df_intermediate= df_intermediate.sort_values(column_name)

# #     #iterate over indexes. Collect indexes for which shorter version exists, so I can drop the less aggregated ones.
# #     for i in range(len(df_intermediate)): 
# #         current_value = df_intermediate.iloc[i-1][column_name]
# #         next_value = df_intermediate.iloc[i][column_name]
        
# #         # Check if the next row's string starts with the current row's string
# #         while next_value.startswith(current_value):
# #             rows_to_drop.add(i)
# #             next_value += 1
    
# #     # Drop the rows from the DataFrame
# #     df_intermediate = df_intermediate.drop(rows_to_drop).reset_index(drop=True)
# #     return df_intermediate


# #function for selecting two levels of disaggregation (when one level isn't available for all series).
# def two_levels(df, column_name, level):
#     df_intermediate= df.loc[df[column_name].str.len().isin([level])] #create dataframe with rows where index has level (e.g.6) characters
#     higher_level = set(df[df[column_name].str.len().isin([level])][column_name].values) #create a set containing the six-digit indexes.
    
#     to_drop=set() #create empty set
#     df_temp = df.loc[df[column_name].str.len().isin([level+1])] #create a dataframe with rows where index has level+1(e.g.7) characters.
#     for i in range(len(df_temp)): #iterate over the rows in the level+1 dataframe.
#         current_value = df_temp.iloc[i][column_name] 
        
#         if current_value[:-1] in higher_level: #if current value is in the set of level indexes...
#             to_drop.add(df_temp.index[i]) #add the row to the to_drop set.
#     df_temp = df_temp.drop(to_drop).reset_index(drop=True) #drop rows from lower level dataframe.
#     df_intermediate = pd.merge(df_intermediate, df_temp, how= "outer") #merge the higher and lower level dataframes
#     df_intermediate = df_intermediate.sort_values(column_name)
#     df_intermediate = df_intermediate.reset_index(drop=True)
#     return df_intermediate

   
# #function for selecting most disaggregated level available.
# def lowest_level(df, column_name):
# # Function to drop rows where the next row consists of the same string plus extra characters
#     df = df.sort_values(column_name)
#     rows_to_drop = set()

#     for i in range(len(df) - 1):
#         current_value = df.iloc[i][column_name]
#         next_value = df.iloc[i + 1][column_name]
#         skipped= 0
        
#         # Check if the next row's string starts with the current row's string
#         if next_value.startswith(current_value):
#             rows_to_drop.add(df.index[i])
            
    
#     # Drop the rows from the DataFrame
#     df_cleaned = df.drop(rows_to_drop).reset_index(drop=True)
    
#     return df_cleaned


In [109]:
#more readable version of intermediate_level filter written by Jacob. To learn from.
# def intermediate_level_alt(df, column_name, level): 
#     df = df.sort_values(column_name)

#     def filter(el): 
#         if len(el) == level: 
#             filter.level.add(el)
#             return True
#         elif len(el) == level + 1 and el[:-1] not in filter.level:
#             return True
#         return False

#     filter.level = set()
#     res = df[df[column_name].apply(filter)]
#     return res

# df_test= filter_df(df_cose,"identity",6)
# df_test

In [175]:
# #DATA CLEANING: part datasets by level of aggregation

# #for commodities/services, select industry, product group, and individual items.
# df_cose_industry = one_level(df_cose, "identity", 5)
# df_cose_group = two_levels(df_cose, "identity", 6)
# df_cose_item = lowest_level(df_cose, "identity")
# df_cose_item = df_cose_item[~df_cose_item.identity.str.contains("wps00000000")] #take out all products index

# #only the lowest level of aggregation is interesting for final demand imo.
# df_fd= lowest_level(df_fd, "identity")

# #for intermediate demand want the stages and each of their inputs.
# df_id_stage = one_level(df_id, "identity", 7)
# df_id_inputs = lowest_level(df_id, "identity")



In [355]:
# cose_data['Date'] = pd.to_datetime(cose_data['Date']).dt.date
# cose_data = cose_data.set_index(['Date'])
# cose_data = cose_data.transpose()

In [357]:
# cose_data = cose_data.dropna(how='all')

In [ ]:

#TO DO:
#download weights and add them to df.
    -download xlsx files
    -create weights dataframe
-create dataframe
-add dates (dec of each year)
-create columns for each code
-import csv file
-(possibly) delete top rows?
-
    -pull weights for index list


#GOALS
-how price categories have changed since 1900s.
-"map of the economy"
-volatility within goods categories and how this relates to headline inflation/category.